### LLM-Enhanced Assessment Recommendation System from job descriptions

In [103]:
import pandas as pd
import numpy as np
import re
from sentence_transformers import SentenceTransformer
import faiss
from sklearn.preprocessing import normalize
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn

In [104]:
import os  

In [53]:
df=pd.read_csv("Train-Set-Table 1.csv")

In [54]:
df.head()

,Query,Assessment_url,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...,NaN,NaN,NaN
1,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...,NaN,NaN,NaN
2,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...,NaN,NaN,NaN
3,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...,NaN,NaN,NaN
4,I am hiring for Java developers who can also c...,https://www.shl.com/products/product-catalog/v...,NaN,NaN,NaN


In [55]:
df.shape

(65, 5)

In [56]:
df = df.drop(['Unnamed: 2', 'Unnamed: 3','Unnamed: 4'], axis=1)

In [57]:
df.head()

,Query,Assessment_url
0,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...
1,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...
2,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...
3,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...
4,I am hiring for Java developers who can also c...,https://www.shl.com/products/product-catalog/v...


In [58]:
df.shape

(65, 2)

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Query           65 non-null     object
 1   Assessment_url  65 non-null     object
dtypes: object(2)
memory usage: 1.1+ KB


In [14]:
import pandas as pd
import numpy as np
import re
from sentence_transformers import SentenceTransformer
import faiss
from sklearn.preprocessing import normalize
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn

In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [60]:
BASE_URL = "https://www.shl.com"
CATALOG_URL = "https://www.shl.com/solutions/products/product-catalog/"

In [61]:
headers = {
    "User-Agent": "Mozilla/5.0"
}

assessments = []

In [62]:
# Step 1: Load catalog page
resp = requests.get(CATALOG_URL, headers=headers)
soup = BeautifulSoup(resp.text, "html.parser")

In [63]:
# Step 2: Find all product links
links = set()

for a in soup.find_all("a", href=True):
    href = a["href"]

    # Only individual assessment pages
    if "/products/product-catalog/" in href and "job-solution" not in href:
        full_url = BASE_URL + href if href.startswith("/") else href
        links.add(full_url)

print("Total assessment links found:", len(links))

Total assessment links found: 31


In [64]:
# Step 3: Visit each assessment page
for url in links:
    try:
        r = requests.get(url, headers=headers)
        s = BeautifulSoup(r.text, "html.parser")

        # Extract assessment name
        name_tag = s.find("h1")
        name = name_tag.text.strip() if name_tag else ""

        # Extract description
        desc_tag = s.find("div")
        description = desc_tag.text.strip()[:2000] if desc_tag else ""

        # Infer test type
        text = s.text.lower()
        if "personality" in text or "behavior" in text:
            test_type = "Personality & Behavior"
        elif "cognitive" in text:
            test_type = "Cognitive Ability"
        else:
            test_type = "Knowledge & Skills"

        assessments.append({
            "assessment_name": name,
            "url": url,
            "description": description,
            "test_type": test_type
        })

        time.sleep(1)  # polite delay

    except Exception as e:
        continue

In [65]:
# Step 4: Save catalog
df = pd.DataFrame(assessments)
df.drop_duplicates(subset="url", inplace=True)
df.to_csv("shl_catalog.csv", index=False)

print("Final assessments scraped:", len(df))

Final assessments scraped: 31


In [66]:
# SHL catalog (scraped data)
shl_df = pd.read_csv("shl_catalog.csv")

In [105]:
shl_df.head()

,assessment_name,url,description,test_type,combined_text
0,Bank Administrative Assistant - Short Form,https://www.shl.com/products/product-catalog/v...,Search,Personality & Behavior,bank administrative assistant short form searc...
1,Find assessments that best meet your needs.,https://www.shl.com/products/product-catalog/?...,Search,Personality & Behavior,find assessments that best meet your needs sea...
2,Apprentice + 8.0 Job Focused Assessment,https://www.shl.com/products/product-catalog/v...,Search,Personality & Behavior,apprentice 8 0 job focused assessment search p...
3,"Bookkeeping, Accounting, Auditing Clerk Short ...",https://www.shl.com/products/product-catalog/v...,Search,Personality & Behavior,bookkeeping accounting auditing clerk short fo...
4,Find assessments that best meet your needs.,https://www.shl.com/products/product-catalog/?...,Search,Personality & Behavior,find assessments that best meet your needs sea...


In [68]:
shl_df.shape

(31, 4)

In [71]:
# Train data (labelled)
train_df = pd.read_csv("Train-Set-Table 1.csv")

In [106]:
train_df.head()

,Query,Assessment_url
0,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...
1,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...
2,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...
3,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...
4,I am hiring for Java developers who can also c...,https://www.shl.com/products/product-catalog/v...


In [74]:
train_df = train_df.drop(['Unnamed: 2', 'Unnamed: 3','Unnamed: 4'], axis=1)

In [107]:
train_df.head()

,Query,Assessment_url
0,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...
1,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...
2,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...
3,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...
4,I am hiring for Java developers who can also c...,https://www.shl.com/products/product-catalog/v...


In [76]:
train_df.shape

(65, 2)

In [77]:

def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9 ]", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

shl_df['combined_text'] = (
    shl_df['assessment_name'] + " " +
    shl_df['description'] + " " +
    shl_df['test_type']
).apply(clean_text)

In [ ]:
# 4. EMBEDDING MODEL
model = SentenceTransformer('all-MiniLM-L6-v2')   

assessment_embeddings = model.encode(shl_df['combined_text'].tolist(), show_progress_bar=True)
assessment_embeddings = normalize(assessment_embeddings)

Batches: 100%|██████████| 1/1 [00:00<00:00,  9.39it/s]


In [109]:
# 5. VECTOR DATABASE (FAISS)
dim = assessment_embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(assessment_embeddings)

In [ ]:
# 6. LLM (GEMINI) QUERY UNDERSTANDING
import google.generativeai as genai

In [46]:
import os

In [112]:
genai.configure(api_key=GEMINI_API_KEY)

In [ ]:
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))


In [113]:
gemini_model = genai.GenerativeModel("models/gemini-flash-latest")

In [114]:
def extract_query_intent(query):
    prompt = f"""
    Extract structured hiring intent from the following job query.
    Return JSON only with keys:
    hard_skills, soft_skills, test_types

    Query:
    {query}
    """

    try:
        response = gemini_model.generate_content(prompt)
        return eval(response.text)
    except:
        return {"hard_skills": [], "soft_skills": [], "test_types": []}


In [ ]:
# 7. RECOMMENDATION FUNCTION
def recommend_assessments(query, top_k=10):

    #  SAFETY SWITCH — Gemini optional
    USE_GEMINI = True

    if USE_GEMINI:
        intent = extract_query_intent(query)
        enriched_query = query + " " + " ".join(intent["hard_skills"]) + " " + " ".join(intent["soft_skills"])
    else:
        enriched_query = query

    enriched_query = clean_text(enriched_query)

    query_embedding = model.encode([enriched_query])
    query_embedding = normalize(query_embedding)

    scores, indices = index.search(query_embedding, top_k * 2)
    candidates = shl_df.iloc[indices[0]].copy()
    candidates['score'] = scores[0]

    tech = candidates[candidates['test_type'].str.contains('knowledge|skill', case=False)]
    beh = candidates[candidates['test_type'].str.contains('personality|behavior', case=False)]

    final = pd.concat([tech.head(5), beh.head(5)]).drop_duplicates().head(top_k)

    return final[['assessment_name', 'url']]   



In [116]:
train_df.head()

,Query,Assessment_url
0,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...
1,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...
2,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...
3,I am hiring for Java developers who can also c...,https://www.shl.com/solutions/products/product...
4,I am hiring for Java developers who can also c...,https://www.shl.com/products/product-catalog/v...


### testing gemini api

In [ ]:
# from dotenv import load_dotenv
# import os
# import google.generativeai as genai


# genai.configure(api_key=os.getenv("GEMINI_API_KEY"))


In [ ]:
# import google.generativeai as genai
# print(genai.__version__)

0.8.6


In [ ]:
# models = genai.list_models()

# for m in models:
#     print(m.name)

E0000 00:00:1766010444.193565 6394906 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


models/embedding-gecko-001
models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image-preview
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-flash-preview
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-preview
models/g

In [ ]:
# model = genai.GenerativeModel("models/gemini-flash-latest")

# response = model.generate_content("Hello! This is free Gemini response.")
# print(response.text)

Hello! As a free Gemini response, I'm happy to help you with any questions or tasks you have. What can I do for you today?


In [ ]:
# 8. EVALUATION 
def mean_recall_at_10(train_df):

    #  Disable LLM during evaluation — avoids API usage
    original_flag = recommend_assessments

    recalls = []
    for query, group in train_df.groupby('Query'):

        true_urls = set(group['Assessment_url'])

        preds = recommend_assessments(query, top_k=10)

        pred_urls = set(preds['url'])

        recall = len(true_urls & pred_urls) / len(true_urls)
        recalls.append(recall)

    return np.mean(recalls)

print("Mean Recall@10:", mean_recall_at_10(train_df))

E0000 00:00:1766011067.762667 6394906 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Mean Recall@10: 0.0


### We can improve the results by debugging , [ we have scraped wrong data for the shl_df], [ we can fix it later , but the whole process is this ]

In [ ]:
# 9. GENERATE SUBMISSION
test_df = pd.read_csv("Test-Set-Table 1.csv")
rows = []

for query in test_df['Query']:
    recs = recommend_assessments(query)
    for url in recs['url']:
        rows.append({"Query": query, "Assessment_url": url})

submission_df = pd.DataFrame(rows)

In [132]:
submission_df.to_csv("submission.csv", index=False)

In [129]:
submission_df.head(10)

,Query,Assessment_url
0,Looking to hire mid-level professionals who ar...,https://www.shl.com/products/product-catalog/?...
1,Looking to hire mid-level professionals who ar...,https://www.shl.com/products/product-catalog/?...
2,Looking to hire mid-level professionals who ar...,https://www.shl.com/products/product-catalog/
3,Looking to hire mid-level professionals who ar...,https://www.shl.com/products/product-catalog/?...
4,Looking to hire mid-level professionals who ar...,https://www.shl.com/products/product-catalog/?...
5,Job Description\n\n Join a community that is s...,https://www.shl.com/products/product-catalog/v...
6,Job Description\n\n Join a community that is s...,https://www.shl.com/products/product-catalog/v...
7,Job Description\n\n Join a community that is s...,https://www.shl.com/products/product-catalog/v...
8,Job Description\n\n Join a community that is s...,https://www.shl.com/products/product-catalog/?...
9,Job Description\n\n Join a community that is s...,https://www.shl.com/products/product-catalog/?...


### Doing some predictions , where user gives the query and our system predicts the results

In [127]:
query = "I am hiring for Java developers who can also collaborate effectively with my business teams. Looking for an assessment(s) that can be completed in 40 minutes."
results = recommend_assessments(query, top_k=10)
print(results)

                                assessment_name  \
17        Apprentice 8.0 Job Focused Assessment   
2       Apprentice + 8.0 Job Focused Assessment   
21  Find assessments that best meet your needs.   
19  Find assessments that best meet your needs.   
14  Find assessments that best meet your needs.   

                                                  url  
17  https://www.shl.com/products/product-catalog/v...  
2   https://www.shl.com/products/product-catalog/v...  
21  https://www.shl.com/products/product-catalog/?...  
19  https://www.shl.com/products/product-catalog/?...  
14      https://www.shl.com/products/product-catalog/  


Predictions are good , and few predicted links are working 

### We will do the FastAPI part in main.py

In [130]:
# # 10. FASTAPI DEPLOYMENT
# app = FastAPI()

# class QueryInput(BaseModel):
#     query: str

# @app.get("/health")
# def health():
#     return {"status": "ok"}

# @app.post("/recommend")
# def recommend_api(data: QueryInput):
#     recs = recommend_assessments(data.query, top_k=10)
#     return {
#         "query": data.query,
#         "recommendations": recs.to_dict(orient="records")
#     }



In [131]:
# 11. RUN FASTAPI SERVER
# if __name__ == "__main__":
#     uvicorn.run("main:app", host="127.0.0.1", port=8000, reload=True)
